Problem statement-
Build a multivariate Linear regression model to predict the medical insurance cost(charges) based on patient information.
1. Perform exploratory data analysis on the dataset
2. Visualize data elements
3. Derive preliminary observations about relationship between the variables (independent, dependent)
4. Perform feature engineering
 - Explore possibility of deriving new features 
 - Transform existing features using techniques like Min-max scaling, One hot encoding(dummy variable creation)
5. Perform feature selection using RFE, VIF, iterative Linear regression model summary (p-value of features coefficients)
6. Perform Residual analysis using the training data
7. Build the final regression equation.


In [2]:
# importing required libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sb

# importing warnings to ingnore any warnings given by the interpreter
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 1. Reading and understanding data

In [3]:
insurance = pd.read_csv('insurance.csv')
insurance.head() # gives the first 5 instances of the table

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
insurance.shape # gives the dimensions of our data
# in this case the data has 1338 rows and 7 columns

(1338, 7)

In [5]:
insurance.dtypes # gives the data types of our data

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [13]:
insurance.describe(include='all').T 
# gives the statistics for the table 
# .T is to get the transpose of the table

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age,1338.0,NaN,NaN,NaN,39.207025,14.04996,18.0,27.0,39.0,51.0,64.0
sex,1338,2,male,676,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bmi,1338.0,NaN,NaN,NaN,30.663397,6.098187,15.96,26.29625,30.4,34.69375,53.13
children,1338.0,NaN,NaN,NaN,1.094918,1.205493,0.0,0.0,1.0,2.0,5.0
smoker,1338,2,no,1064,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region,1338,4,southeast,364,NaN,NaN,NaN,NaN,NaN,NaN,NaN
charges,1338.0,NaN,NaN,NaN,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801
